# Lecture 2

## How is efficiency of an IR system measured?

Efficiency is measured in 5 ways:

1. Indexing time : Time taken to index a given set of documents.
2. Indexing space: Space taken in memory, during indexing.
3. Indexing storage: Disk space taken to store the index.
4. Query latency : Time taken to execute the query
5. Query Throughput : Number of queries executed in unit time.

## How is an AND query executed?

The AND query works by intersecting posting lists. 

term1 -> d1, d4, d6, d7

term2 -> d3, d5, d9, d10, d11, d108


The resulting AND query for term1 and term2 will return only d4

## How do we intersect posting lists?

The algorithm used to intersect posting lists is also called the merge algorithm. 

Pseudocode:

```
# Input is two posting lists sorted in ascending order.
merge(l_a, l_b):
    ans = []
    # Init the pointers to the list
    p1 = 0
    p2 = 0
    while p1 < len(l_a) and p2 < len(l_b):
        if l_a[p1] == l_b[p2]:
            ans.append(l_a[p1])
            p1 += 1
            p2 += 1
        elif l_a[p1] < l_b[p2]:
            p1 += 1
        else:
            p2 += 1
    return ans
```

What is happening in the algorithm is as follows.

We initialise two pointers to point to the first element of two posting lists.

As long as the pointers haven't crossed the list, ie, still point to elements within the list, we continue in the while loop. 

if the elements of both the lists are equal, then we increment both pointers.

If not, we increment the pointer that points to the smaller element in the given list.

## What is the running time of the merge algorithm?

The running time is $ O(m * n) $

Where $m, n$ are length of the two lists respectively.

## How can we reduce the running time of the merge algorithm?

We use something called skip pointers. It is shown in the image below.

<img src="Skip_Pointers.png">

## How are skip pointers used?

When a skip pointer is used in a posting list, the merge algorithm changes as follows.

Instead of comparing with the elements in the posting lists, we check if there is a skip pointer from the current element. If the skip pointer points to an element that is smaller than the element on the other list, then we can jump ahead to the skip pointer's location instead of incrementing and checking. 

## What are the challenges of using skip pointers?

The challenges are:

1. More skip distance -> few successful skips
2. Short skip distance -> more successful skips, but more comparisions. 

## How are skips placed?

Skips are placed based on a heuristic. A good idea is to use $\sqrt{L}$

Where $L$ is the length of the posting list.

One issue with this is that if L changes, then the skip pointers will be garbled. 

## What are phrase queries? 

Queries where instead of treating each query term as individual, a group of terms, like **Stanford University** is considered to be a single search term. 

This is important because of the information need of the user. 

## How do we handle phrase queries?

We handle phrase queries by using two approaches: 

1. Phrase indexes
2. Positional indexes

## What are phrase indexes?

Instead of using a single tokens are indexes, we use a group of tokens as indexes. Say, if we're using bi-word indexes, then we index using two terms instead of just one. 

## What are n-grams?

Phrase indexing where a group of n terms are considered to be an index.

Bi-word is when two terms are indexed together.

Tri-word is when three words are indexed together.

## How do we use n-grams?

We index for all sequential combinations of n terms.

## How does a query with 4 terms work when using a bi-word index?

When a query has 4 terms and we have bi-gram indexing, **Stanford Uni Palo Alto**, we do the following:

1. Split the query into pairs of terms, **[Stanford Uni], [Uni Palo], [Palo Alto]** 

    We then perform a boolean AND query on the sets of words. As
    **[Stanford Uni] AND [Uni Palo] AND [Palo Alto]**

2. We then post-process the results to filter out false-positives.

## What are the benefits of using phrase indexes? 

If we use phrase indexes, then the chances of getting false positives reduces. Especially if we use tri-word.

## What are the cons of using phrase indexes?

Phrase indexes tend to make the index sizes large. This is a problem if we have to index on a large corpus and if the corpus keeps changing. 

## What is position indexing?

Poisitional indexing is when along with storing the document where the term was found, the position where the term was found. 

## How does proximity search work?

When we use positional indexing, we are in a position where we can do the following query.

**Employment \4 Place** where it means that the term place has to be 4 places after employment. 

## What is the benefit of positional indexing?

1. Saves space when compared to phrase indexing.
2. Allows us to do proximity searches. 

## What are combinational schemes?

Instead of using all bi-words, we can instead index on only common bi-words. This along with positional indexing is called a combinational scheme.

## How do we find all documents that contain terms that begin with the word **mon**?

The vocabulary has to be stored as a tree.

Let's represent all words that begin with the word mon as mon*. This is a wildcard query. 

It is possible to find all terms that begin with mon* if we use a tree to store the indexes.

If a tree is used, the search quickly becomes finding all terms that are in the range mon <= moo

## How do we find all documents that end with mon?

We follow the same principle as above. Instead of storing terms as they are in the tree, we store reversed terms in the tree. 

When searching, we then search for terms that begin with *mon* in the reversed tree.

## How does permutex indexing help us with wildcard queries? 

Permutex indexing helps when the wildcard is in the middle of the term.

Example, he*lo?

We append a $ symbol to the query and rotate it such that the * is at the end of the query.

he*lo -> he*lo$ -> lo$he*

We then map this rotated term lo$he* to hello

## What are k-grams? Use bi-gram to explain.

Instead of mapping words, we map k sequences of characters. The special symbol $ is used to denote start and end of a word.

So a word like $hello \rightarrow {$h, he, el, ll, lo, o$}$

Each of these k-gram terms are then mapped, in another inverted index, to words that they appear in.

Ex: 

$m -> melt, mars, moon, etc

ma -> madden, mars, mash, mast

## How are wildcard queries processed when using k-grams?

Suppose we're given a query of the form mon*

We break the query into bi-grams(assuming bi-gram indexing) and the query term is now

mon* -> {$m, mo, on}

We then run a boolean AND query for the above bi-grams.

Post filtering has to be performed on the results of the above query.

## What are the uses of spell correction?

Uses:
1. Correcting document terms
2. Correcting user query

## What are the two forms of spell correction:

1. Isolated form : The word is corrected in isolation, ie, the surrounding words are not looked at.
2. Context-sensitive form : The words surrounding the incorrect word is also looked at before spell-correction. 

## What is edit distance?

Given two words, how many edits is needed to convert one word to the other word, by only using insert, replace or delete. 

## How is Levenshtein distance calculated?

The algo for Levenshtein distance is :

```
if s1[i] == s2[j]:
    dist = min(
        m[i-1, j-1],
        m[i - 1, j] + 1,
        m[i, j - 1] + 1
    )
else:
    dist = min(
        m[i-1, j-1] + 1,
        m[i - 1, j] + 1,
        m[i, j - 1] + 1
    )
```

The cells corresponding to i=0 or j=0 have values equal to their non-zero index. 

<img src='Levenshtein_Distance.png'>